In [2]:
# Define your model
from torchvision.models import resnet18, resnet50, ResNet50_Weights, ResNet18_Weights
from torchcam.methods import SmoothGradCAMpp
from torchvision.transforms.functional import normalize, resize
from torchvision.models import resnet18
from moviepy.editor import *
from crop import process_frame_custom
import numpy as np
import torch


In [3]:
aspect_ratio = (9, 16)
input_video = "/Users/adiprabs/sample.mp4"

# Output video file
output_video = "output_video13.mp4"

# Desired aspect ratio
    # Example aspect ratio

# Function to crop and adjust center of the frame

# Open input video
video_clip = VideoFileClip(input_video)
audio_clip = video_clip.audio
frames = video_clip.iter_frames()


processed_frames = []
height, width = video_clip.get_frame(1).shape[:2]
crop_height = height
crop_width = int(height*aspect_ratio[0]/aspect_ratio[1])
center = int(width/2)
i = 0
n_frames = video_clip.reader.nframes
model = resnet18(weights=ResNet18_Weights.DEFAULT).eval()
frequency = 3 # Adjusts after how many frames the center is readjusted based on the attention model
counter = frequency
transition_factor = 0.8 # The higher, the faster it moves between points of interest, must be positive
if transition_factor <= 0:
    raise Exception("MUST be positive")

with SmoothGradCAMpp(model) as cam_extractor:
    for frame in frames:
        if counter == frequency:
            counter = 0
            frame1 = torch.from_numpy(np.array(frame)).permute(2, 0, 1)

            # Preprocess it for your chosen model
            input_tensor = normalize(resize(frame1, (224, 224)) / 255., [0.485, 0.456, 0.406], [0.229, 0.224, 0.225])

            # Preprocess your data and feed it to the model
            out = model(input_tensor.unsqueeze(0))
            # Retrieve the CAM by passing the class index and the model output
            activation_map = cam_extractor(out.squeeze(0).argmax().item(), out)
            
            b = np.array(activation_map[0][0])
            sumx = 0
            sumy = 0
            total_sum = np.sum(b) # Add a small constant to avoid division by zero
            for x in range(len(b)):
                for y in range(len(b[0])):
                    sumx += b[x, y] * (x + 0)
                    sumy += b[x, y] * (y + 0)
            avgx = sumx / np.sum(b)
            avgy = sumy / np.sum(b)
            average_coords = (avgx, avgy)
            # if (width * avgx / 7) > center:
            #     center += 5
            # if (width * avgx / 7) < center:
            #     center -= 5
            center = int(((1/transition_factor)*center + (width * avgx / 7))/(1 + 1/transition_factor))
        processed_frames.append(process_frame_custom(frame, center))
        counter += 1


output_clip = ImageSequenceClip(processed_frames, fps=video_clip.fps)
output_clip = output_clip.set_audio(audio_clip)
output_clip.write_videofile(output_video,fps=video_clip.fps, codec="mpeg4", audio_codec="aac")
print("Done")

  warnings.warn("Warning: in file %s, "%(self.filename)+



Moviepy - Building video output_video13.mp4.
MoviePy - Writing audio in output_video13TEMP_MPY_wvf_snd.mp4


MoviePy - Done.
Moviepy - Writing video output_video13.mp4



Moviepy - Done !
Moviepy - video ready output_video13.mp4
